# Displaying marker pairs as network

<div id="toc"></div>

## Neccessary Imports

In [10]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [11]:
import sys
code = "./../../code/"
data = "./../../data/"
sys.path.append(code)
import pandas
import pypairs as pairs
from sklearn.preprocessing import QuantileTransformer
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import numpy as np
from pathlib import Path
from tqdm import tqdm_notebook as tqdm
import helper
from itertools import chain
import networkx as nx

init_notebook_mode(connected=True)

## Load oscope marker pairs

In [12]:
oscope_marker_pairs = helper.load_ocope_marker(data, fraction=0.6)

[__set_matrix] Original Matrix 'x' has shape 19084 x 247
[__set_matrix] Removed 16689 genes that were not in 'subset_genes'. 2395 genes remaining.
[__set_matrix] Removed 61 genes that were not expressed in any samples. 2334 genes remaining.
[__set_matrix] Removed 0 samples that were not annotated in 'phases'. 247 samples remaining.
[__set_matrix] Matrix truncation done. Working with 2334 genes for 247 samples.
[sandbag] Identifying marker pairs...Processing in parallel with 10 processes...
 Done!
[sandbag] Identified 8146 marker pairs (phase: count): {'G1': 2575, 'S': 4101, 'G2M': 1470}


## Construct network from G2M marker pairs

### Adding Nodes and Edges from pairs (A,B) => (A)->(B)

In [13]:
# Load matrix
oscope_gencounts = pandas.read_csv(Path(data + "GSE64016_H1andFUCCI_normalized_EC_human.csv"))

# Set index right
oscope_gencounts.set_index("Unnamed: 0", inplace=True)

# Subset sorted
oscope_gencounts_sorted = oscope_gencounts.iloc[:, [oscope_gencounts.columns.get_loc(c) for c in oscope_gencounts.columns if "G1_" in c or "G2_" in c or "S_" in c]]


genes = oscope_gencounts_sorted.index.tolist()

marker_edges = []
for marker in oscope_marker_pairs["G2M"]:
    marker_edges.append((genes.index(marker[0]), genes.index(marker[1])))


marker_edges

[(12772, 11412),
 (18544, 15948),
 (12094, 1509),
 (9334, 6274),
 (9334, 5846),
 (9334, 7262),
 (10848, 17807),
 (9849, 15364),
 (11680, 16129),
 (16334, 16129),
 (9849, 16129),
 (13025, 17307),
 (9006, 15913),
 (13025, 15913),
 (9006, 10400),
 (6886, 1817),
 (10848, 4169),
 (99, 4169),
 (8934, 5078),
 (12094, 5078),
 (6690, 5078),
 (6886, 8995),
 (12772, 15762),
 (9334, 15762),
 (9849, 15762),
 (18265, 14526),
 (9006, 5279),
 (9006, 9836),
 (9445, 9836),
 (6886, 14792),
 (11680, 9203),
 (5656, 9203),
 (9334, 16510),
 (9334, 10343),
 (4581, 1066),
 (2125, 12563),
 (9334, 2125),
 (2125, 146),
 (14416, 2799),
 (12784, 2799),
 (15385, 2799),
 (9849, 4296),
 (9334, 11884),
 (18442, 1393),
 (9323, 1393),
 (724, 1393),
 (3499, 1393),
 (8934, 1393),
 (43, 1393),
 (15335, 1393),
 (19077, 1393),
 (12094, 1393),
 (11680, 9875),
 (9849, 9875),
 (9334, 11770),
 (12951, 16621),
 (9849, 12951),
 (10848, 17157),
 (8496, 17157),
 (12784, 17157),
 (9334, 1850),
 (744, 2017),
 (5656, 744),
 (10806, 1441

In [14]:
net_G2M=nx.DiGraph()

net_G2M.add_edges_from(marker_edges)

### Position on plot

In [15]:
pos=nx.spring_layout(net_G2M)

dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

pos

{21: array([-0.97067976,  0.11241949]),
 27: array([0.57628816, 0.34959951]),
 37: array([0.55178386, 0.74100631]),
 43: array([-0.16415732,  0.08261263]),
 79: array([-0.8615669 ,  0.40461448]),
 80: array([-0.83196241,  0.43416432]),
 99: array([-0.05694951, -0.28225255]),
 114: array([0.95807552, 0.20455635]),
 146: array([ 0.91983986, -0.19865361]),
 225: array([-0.18268165, -0.20626184]),
 233: array([-0.90627944, -0.46009552]),
 286: array([0.15500392, 0.75839579]),
 296: array([0.92668688, 0.37392318]),
 350: array([-0.84281576, -0.30803025]),
 354: array([ 0.86466348, -0.11546593]),
 377: array([-0.19273145, -0.91664559]),
 390: array([-0.89492953, -0.02494552]),
 405: array([0.25143209, 0.01062035]),
 454: array([-0.79834074,  0.2461306 ]),
 462: array([-0.75378239, -0.70561558]),
 481: array([-0.01464985,  0.92304748]),
 517: array([-0.05335184,  0.45324409]),
 536: array([ 0.71962136, -0.61584854]),
 543: array([ 0.52191895, -0.16136606]),
 544: array([-0.46608919, -0.796763

## Plot, color according to degree of connectivity

In [16]:
p=nx.single_source_shortest_path_length(net_G2M,ncenter)

edge_trace = go.Scatter(
    x=[],
    y=[],
    line=go.Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in net_G2M.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=go.Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in net_G2M.nodes():
    x, y = pos[node]
    node_trace['x'].append(x)
    node_trace['y'].append(y)
    
for node in net_G2M.nodes:
    node_trace['marker']['color'].append(net_G2M.degree(node))
    node_info = '# of connections: '+str(net_G2M.degree(node))
    node_trace['text'].append(node_info)

fig = go.Figure(data=go.Data([edge_trace, node_trace]),
             layout=go.Layout(
                title='Network of Marker pairs',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=go.XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=go.YAxis(showgrid=False, zeroline=False, showticklabels=False)))

iplot(fig, filename='networkx')